<a href="https://colab.research.google.com/github/Atomix77/IMLO-IP/blob/main/IMLO_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data



In [22]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

trainingTransform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(20),
      transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),])

testTransform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),])

trainingData = datasets.Flowers102(
    root = "./datasets",
    split = "train",
    transform = trainingTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

validationData = datasets.Flowers102(
    root = "./datasets",
    split = "val",
    transform = testTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

testData = datasets.Flowers102(
    root = "./datasets",
    split = "test",
    transform = testTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

# Dataloaders

In [23]:
batchSize = 64

trainingDataloader = DataLoader(trainingData, batchSize, shuffle = True, num_workers = 6)
validationDataloader = DataLoader(validationData, batchSize, shuffle = False, num_workers = 6)
testDataloader = DataLoader(testData, batchSize, shuffle = False, num_workers = 6)

# Get device

In [24]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Create Aritecture

In [25]:
class NeuralNetwork(nn.Module):
  def __init__(self, classAmount):
    super(NeuralNetwork, self).__init__()
    self.convStack =  nn.Sequential(
      nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 2, stride = 2),
      nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 2, stride = 2),
      nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 2, stride = 2),
      nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1),
      nn.BatchNorm2d(256),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 2, stride = 2),
      nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1),
      nn.BatchNorm2d(512),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 2, stride = 2),
      nn.Dropout(0.5)
      )
    self.classifier = nn.Sequential(
      nn.Linear(32 * 28 * 28, 1024),
      nn.ReLU(),
      nn.Dropout(0.5),
      nn.Linear(1024, classAmount)
      )
    

  def forward(self, x):
    x = self.convStack(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x
  
  
model = NeuralNetwork(102).to(device)

# Testing

In [26]:
def testModel(dataloader, model, lossFunction):
  model.eval()
  total = 0
  correct = 0
  testLoss = 0
  
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      total += y.size(0)
      testLoss += lossFunction(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  
  testLoss = testLoss/total
  correct = (correct/total) * 100
  print(f"Testing: Accuracy: {(correct):>0.1f}%, Avg loss: {testLoss:>8f} \n")
  return testLoss, correct

# Training

In [27]:
earlyStopCount = 5
def trainingModel(dataloader, model, lossFunction, optimizer, epochs):
  bestValLoss = float('inf')
  bestValAccuracy = 0
  waitCounter = 0

  for epoch in range(epochs):
    model.train()
    print(f'Epoch{epoch+1}:')
    currentLoss = 0.0
    correct = 0
    total = 0
    testLoss = 0
    
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      optimizer.zero_grad()
      
      pred = model(X)
      loss = lossFunction(pred, y)

      loss.backward()
      optimizer.step()

      currentLoss += loss.item()

      total += y.size(0)
      testLoss += lossFunction(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    testLoss = testLoss/total
    correct = correct/total
    print(f'Training: Accuracy {(100*correct):>0.1f}%, Loss: {currentLoss / len(dataloader):.5f}, Test Losss: {testLoss:.5f}')
    
    valLoss, valAccuracy = testModel(validationDataloader, model, lossFunction)
    if (valAccuracy < bestValAccuracy):
      waitCounter += 1
    else:
      bestValLoss = valLoss
      bestValAccuracy = valAccuracy
      waitCounter = 0

    # if waitCounter > earlyStopCount:
    #   print(f'Validation has not improved for {earlyStopCount} epochs.')
    #   break
  print(f'Best Accuracy: {bestValAccuracy}. Best Loss: {bestValLoss}')

In [28]:
learningRate = 0.0001
batchSize = 128
epochs = 100

lossFunction = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)

trainingModel(trainingDataloader, model, lossFunction, optimizer, epochs)
testModel(testDataloader, model, lossFunction)
print("Finished")

Epoch1:
Training: Accuracy 1.8%, Loss: 4.87664, Test Losss: 0.07650
Testing: Accuracy: 1.6%, Avg loss: 0.071814 

Epoch2:
Training: Accuracy 4.4%, Loss: 4.34351, Test Losss: 0.06813
Testing: Accuracy: 8.3%, Avg loss: 0.065630 

Epoch3:
Training: Accuracy 8.1%, Loss: 4.05523, Test Losss: 0.06361
Testing: Accuracy: 13.9%, Avg loss: 0.059956 

Epoch4:
Training: Accuracy 11.6%, Loss: 3.80884, Test Losss: 0.05975
Testing: Accuracy: 18.6%, Avg loss: 0.056605 

Epoch5:
Training: Accuracy 15.9%, Loss: 3.51966, Test Losss: 0.05521
Testing: Accuracy: 21.1%, Avg loss: 0.053515 

Epoch6:
Training: Accuracy 21.8%, Loss: 3.32939, Test Losss: 0.05223
Testing: Accuracy: 25.6%, Avg loss: 0.051287 

Epoch7:
Training: Accuracy 22.4%, Loss: 3.14978, Test Losss: 0.04941
Testing: Accuracy: 26.6%, Avg loss: 0.048756 

Epoch8:
Training: Accuracy 25.6%, Loss: 3.02928, Test Losss: 0.04752
Testing: Accuracy: 29.6%, Avg loss: 0.047305 

Epoch9:
Training: Accuracy 29.3%, Loss: 2.81252, Test Losss: 0.04412
Testing:

c:\Users\tomha\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Testing: Accuracy: 47.4%, Avg loss: 0.038937 

Finished


Test - Accuracy: 47.2%, Avg Loss: 0.038937

# Saving the model

In [29]:
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model, 'model.pth')

# Loading the model

In [30]:
# model = models.vgg16()
# model.load_state_dict(torch.load('model.pth'))
# model.eval()